In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler,QuantileTransformer
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import GammaRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor,ExtraTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor,HistGradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from lazypredict.Supervised import LazyRegressor
from sklearn.discriminant_analysis import TransformerMixin

# Analysis

In [2]:
data=pd.read_csv('train.csv')
data

,Id,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,1.06,Ideal,I,SI2,61.80,57.00,4270,6.57,6.60,4.07
1,2,1.51,Premium,G,VVS2,60.90,58.00,15164,7.38,7.42,4.51
2,3,0.32,Ideal,F,VS2,61.30,56.00,828,4.43,4.41,2.71
3,4,0.53,Ideal,G,VS2,61.20,56.00,1577,5.19,5.22,3.19
4,5,0.70,Premium,H,VVS2,61.00,57.00,2596,5.76,5.72,3.50
...,...,...,...,...,...,...,...,...,...,...,...
43147,43148,0.52,Ideal,E,VS2,61.50,56.00,1760,5.16,5.18,3.18
43148,43149,0.72,Very Good,D,VS2,62.10,59.00,3016,5.70,5.73,3.55
43149,43150,0.44,Premium,I,VVS1,61.50,58.00,990,4.95,4.87,3.02
43150,43151,0.31,Premium,E,VS1,60.20,58.00,734,4.38,4.43,2.65


In [3]:
data['volume'] = data['x'] * data['y'] * data['z']

# Calculate density
data['density'] = data['carat'] / data['volume']

# Calculate surface area
data['surface_area'] = 2 * (data['x'] * data['y'] + data['x'] * data['z'] + data['y'] * data['z'])

# Calculate aspect ratio
data['aspect_ratio'] = data['x'] / data['y']

# Calculate total depth percentage
data['total_depth_percentage'] = data['depth'] / data['z']

# Calculate table percentage
data['table_percentage'] = data['table'] / data[['x', 'y']].mean(axis=1)

# Calculate symmetry
data['symmetry'] = data['x'] / data['y'] - data['y'] / data['z']


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43152 entries, 0 to 43151
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Id                      43152 non-null  int64  
 1   carat                   43152 non-null  float64
 2   cut                     43152 non-null  object 
 3   color                   43152 non-null  object 
 4   clarity                 43152 non-null  object 
 5   depth                   43152 non-null  float64
 6   table                   43152 non-null  float64
 7   price                   43152 non-null  int64  
 8   x                       43152 non-null  float64
 9   y                       43152 non-null  float64
 10  z                       43152 non-null  float64
 11  volume                  43152 non-null  float64
 12  density                 43152 non-null  float64
 13  surface_area            43152 non-null  float64
 14  aspect_ratio            43148 non-null

In [5]:
data.describe()

,Id,carat,depth,table,price,x,y,z,volume,density,surface_area,aspect_ratio,total_depth_percentage,table_percentage,symmetry
count,43152.00,43152.00,43152.00,43152.00,43152.00,43152.00,43152.00,43152.00,43152.00,43152.00,43152.00,43148.00,43152.00,43152.00,43148.00
mean,21576.50,0.80,61.75,57.46,3929.49,5.73,5.74,3.54,129.85,inf,152.46,1.00,inf,inf,-inf
std,12457.05,0.47,1.44,2.23,3985.53,1.12,1.15,0.71,78.62,NaN,60.36,0.01,NaN,NaN,NaN
min,1.00,0.20,43.00,43.00,326.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.94,1.70,-inf
25%,10788.75,0.40,61.00,56.00,947.75,4.71,4.72,2.91,65.14,0.01,99.78,0.99,15.30,8.78,-0.64
50%,21576.50,0.70,61.80,57.00,2401.00,5.70,5.71,3.53,114.83,0.01,145.65,1.00,17.54,10.17,-0.62
75%,32364.25,1.04,62.50,59.00,5312.00,6.54,6.54,4.04,170.84,0.01,189.87,1.01,21.20,12.11,-0.60
max,43152.00,5.01,79.00,95.00,18823.00,10.74,58.90,31.80,3840.60,inf,2032.88,1.62,inf,inf,0.83


In [6]:
data.isnull().sum()

Id                        0
carat                     0
cut                       0
color                     0
clarity                   0
depth                     0
table                     0
price                     0
x                         0
y                         0
z                         0
volume                    0
density                   0
surface_area              0
aspect_ratio              4
total_depth_percentage    0
table_percentage          0
symmetry                  4
dtype: int64

In [7]:
data.duplicated().sum()

0

In [8]:
data=data.drop(['Id'],axis=1)

In [9]:
data.corr()

ValueError: could not convert string to float: 'Ideal'

In [ ]:
data['xyz']=data['x']*data['y']*data['z']
data

In [ ]:
# this one for showing boxplots

numeric_cols = data.select_dtypes(include=['number']).columns
for col in numeric_cols:
    if col not in 'price':
        plt.figure(figsize=(6, 4))
        sns.boxplot(x=data[col])
        plt.title(f'Boxplot for {col}')
        plt.show()

# Feature Engineering

In [ ]:
def impute_outliers_with_bounds(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return series.apply(lambda x: lower_bound if x < lower_bound else upper_bound if x > upper_bound else x)
num_features = data.select_dtypes(include=[np.number]).columns
for feature in num_features:
    if feature not in 'price':
        data[feature] = impute_outliers_with_bounds(data[feature])

In [ ]:
numeric_cols = data.select_dtypes(include=['number']).columns
for col in numeric_cols:
    plt.figure(figsize=(6, 4))
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot for {col}')
    plt.show()

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
data['color'] = label_encoder.fit_transform(data['color'])
data['cut'] = label_encoder.fit_transform(data['cut'])
data['clarity'] = label_encoder.fit_transform(data['clarity'])

In [ ]:
data

In [ ]:
ss=StandardScaler()
mm=MinMaxScaler()
rs=RobustScaler()
qt=QuantileTransformer()
ft=FunctionTransformer()

In [ ]:
data['carat'] =qt.fit_transform(data[['carat']])
data['depth'] =qt.fit_transform(data[['depth']])
data['table'] =qt.fit_transform(data[['table']])
data['x'] =qt.fit_transform(data[['x']])
data['y'] =qt.fit_transform(data[['y']])
data['z'] =qt.fit_transform(data[['z']])

In [ ]:
data

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price

# Building Models

In [ ]:
# never run this

regressors = [
    LinearRegression(),
    GammaRegressor(),
    SVR(),
    DecisionTreeRegressor(),
    RandomForestRegressor(),
    MLPRegressor(),
    AdaBoostRegressor(),
    PassiveAggressiveRegressor(),
    KNeighborsRegressor(),
    HistGradientBoostingRegressor(),
    LGBMRegressor(),
    ExtraTreeRegressor(),
    XGBRegressor()
]
n=5
acclist=[]
acclist1=[]
accs=dict()
accs1=dict()
for regressor in regressors:
    acclist=[]
    acclist1=[]
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        acc = r2_score(y_test, y_pred)
        acc1 = np.sqrt(mean_squared_error(y_test, y_pred))
        acclist.append(acc)
        acclist1.append(acc1)
    accs[regressor]=[np.mean(acclist),np.std(acclist)]
    accs1[regressor]=[np.mean(acclist1),np.std(acclist1)]
display(accs)
display(accs1)

In [ ]:
cvs1=cross_val_score(dtr,X_train,y_train, cv=10)
display(cvs1)
(cvs1.mean(),cvs1.std())

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2,random_state=42)
rfr=RandomForestRegressor()
rfr.fit(X_train,y_train)
y_pred = rfr.predict(X_test)
display(r2_score(y_pred,y_test))
display(np.sqrt(mean_squared_error(y_pred,y_test)))

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
X_train1, X_test1, y_train1, y_test1 = train_test_split(x, y, test_size = 0.2)
lgpm=LGBMRegressor()
lgpm.fit(X_train1,y_train1)
y_pred4 = lgpm.predict(X_test1)
display(r2_score(y_pred4,y_test1))
display(np.sqrt(mean_squared_error(y_pred4,y_test1)))

In [ ]:
# NEVER EVER RUN THIS UNLESS YOU ARE 100 PERCENT SURE

param_grid = {
    'n_estimators': np.arange(600, 700, 20),
    'learning_rate': np.arange(0.04, 0.045, 0.001),
    'max_depth': np.arange(5, 8, 1),
    'num_leaves': np.arange(20, 41, 5), 
    'min_child_samples': np.arange(10, 31, 5), 
    'reg_alpha': [0, 0.1, 0.5, 1.0], 
    'reg_lambda': [0, 0.1, 0.5, 1.0] 
}
lgbm = LGBMRegressor()
grid_search = GridSearchCV(lgbm, param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train1, y_train1)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

In [ ]:
x3=data.drop(['price'],axis=1)
y3=data.price
X_train2, X_test2, y_train2, y_test2 = train_test_split(x3, y3, test_size = 0.2)
hg=HistGradientBoostingRegressor()
hg.fit(X_train2,y_train2)
y_pred5 = hg.predict(X_test2)
display(r2_score(y_pred5,y_test2))
display(np.sqrt(mean_squared_error(y_pred5,y_test2)))

In [ ]:
data

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
hgbr=HistGradientBoostingRegressor()
hgbr.fit(x,y)

# Testing

In [ ]:
test=pd.read_csv('test.csv')
test

In [ ]:
test['color'] = label_encoder.fit_transform(test['color'])
test['cut'] = label_encoder.fit_transform(test['cut'])
test['clarity'] = label_encoder.fit_transform(test['clarity'])
test['carat'] =rs.fit_transform(test[['carat']])
test['depth'] =rs.fit_transform(test[['depth']])
test['table'] =rs.fit_transform(test[['table']])
test['x'] =rs.fit_transform(test[['x']])
test['y'] =rs.fit_transform(test[['y']])
test['z'] =rs.fit_transform(test[['z']])

In [ ]:
test.drop('Id',axis=1)

In [ ]:
test['price']=hgbr.predict(test.drop('Id',axis=1))

In [ ]:
test.rename(columns={'Id': 'id'}, inplace=True)

In [ ]:
test1=test[['id','price']]
test1

In [ ]:
test1.to_csv('final_sub.csv',index=False)

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
lgpm=LGBMRegressor()
lgpm.fit(x,y)

In [ ]:
test['price']=lgpm.predict(test.drop(['id','price'],axis=1))

In [ ]:
test2=test[['id','price']]
test2

In [ ]:
test2.to_csv('final_sub1.csv',index=False)

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
xgbr=XGBRegressor()
xgbr.fit(x,y)
test['price']=xgbr.predict(test.drop(['id','price'],axis=1))
test3=test[['id','price']]
test3

In [ ]:
test3.to_csv('final_sub2.csv',index=False)

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
hgbr=HistGradientBoostingRegressor()
hgbr.fit(x,y)

In [ ]:
test=pd.read_csv('test.csv')
test

In [ ]:
test['xyz']=test['x']*test['y']*test['z']

In [ ]:
num_features = test.select_dtypes(include=[np.number]).columns
for feature in num_features:
    if feature not in 'price':
        data[feature] = impute_outliers_with_bounds(data[feature])

In [ ]:
test['color'] = label_encoder.fit_transform(test['color'])
test['cut'] = label_encoder.fit_transform(test['cut'])
test['clarity'] = label_encoder.fit_transform(test['clarity'])
test['carat'] =qt.fit_transform(test[['carat']])
test['depth'] =qt.fit_transform(test[['depth']])
test['table'] =qt.fit_transform(test[['table']])
test['x'] =qt.fit_transform(test[['x']])
test['y'] =qt.fit_transform(test[['y']])
test['z'] =qt.fit_transform(test[['z']])

In [ ]:
test['price']=hgbr.predict(test.drop('Id',axis=1))

In [ ]:
test.rename(columns={'Id': 'id'}, inplace=True)

In [ ]:
test4=test[['id','price']]
test4

In [ ]:
test4.to_csv('final_sub4.csv',index=False)

In [ ]:
x=data.drop(['price'],axis=1)
y=data.price
xgbr=XGBRegressor()
xgbr.fit(x,y)
test['price']=xgbr.predict(test.drop(['id','price'],axis=1))
test6=test[['id','price']]
test6

In [ ]:
test

In [ ]:
x=data.drop(['price','Id'],axis=1)
y=data.price
lgbm=LGBMRegressor()
lgbm.fit(x,y)
test['price']=lgbm.predict(test.drop(['Id'],axis=1))
test.rename(columns={'Id': 'id'}, inplace=True)
test6=test[['id','price']]
test6

In [ ]:
x=data.drop(['price','Id'],axis=1)
y=data.price
rfr=RandomForestRegressor()
rfr.fit(x,y)
test['price']=rfr.predict(test.drop(['Id'],axis=1))
test.rename(columns={'Id': 'id'}, inplace=True)
test6=test[['id','price']]
test6

In [ ]:
test6.to_csv('final_sub8.csv',index=False)

In [ ]:
x=data.drop(['Id','price'],axis=1)
y=data.price
hgbr=HistGradientBoostingRegressor()
hgbr.fit(x,y)
test['price']=hgbr.predict(test.drop(['id','price'],axis=1))
test.rename(columns={'Id': 'id'}, inplace=True)
test7=test[['id','price']]
test7

In [ ]:
test7.to_csv('final_sub6.csv',index=False)

In [ ]:
x=data.drop(['price','Id'],axis=1)
y=data.price
lgbm=LGBMRegressor(learning_rate= 0.042,max_depth= 7,n_estimators= 640)
lgbm.fit(x,y)
test['price']=lgbm.predict(test.drop(['id','price'],axis=1))
test.rename(columns={'Id': 'id'}, inplace=True)
test6=test[['id','price']]
test6

In [ ]:
test6.to_csv('final_sub14.csv',index=False)